In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [3]:
data_2022_updated = pd.read_csv("./Data/BEST_2022_nfl_data_play_with_scores_and_HomeWon (1).csv")
data_2023_updated = pd.read_csv("./Data/BEST_2023_nfl_data_play_with_scores_and_HomeWon (1).csv")

In [4]:
upcomingGames = pd.read_csv("./Data/BEST New_Upcoming_Schedule (1).csv")

In [5]:
all_data = pd.concat([data_2022_updated, data_2023_updated])

In [6]:
all_data.head()

,GameId,Date,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsNoPlay,PenaltyType,PenaltyYards,Week,Visitor,VisitorScore,Home,HomeScore,OT,HomeWon
0,2022091106,09/11/2022,2,1,17,MIA,NE,2,10,55,...,0,NaN,0,Week 1,NE,7.0,MIA,20.0,NaN,True
1,2022091106,09/11/2022,3,7,4,NE,MIA,4,1,72,...,1,ILLEGAL CONTACT,5,Week 1,NE,7.0,MIA,20.0,NaN,True
2,2022091106,09/11/2022,3,6,57,NE,MIA,1,10,77,...,0,NaN,0,Week 1,NE,7.0,MIA,20.0,NaN,True
3,2022091106,09/11/2022,3,6,17,NE,MIA,2,10,77,...,0,NaN,0,Week 1,NE,7.0,MIA,20.0,NaN,True
4,2022091106,09/11/2022,4,9,46,NE,MIA,4,3,100,...,0,OFFENSIVE HOLDING,0,Week 1,NE,7.0,MIA,20.0,NaN,True


In [7]:
all_data.tail()

,GameId,Date,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsNoPlay,PenaltyType,PenaltyYards,Week,Visitor,VisitorScore,Home,HomeScore,OT,HomeWon
12004,2023091002,09/10/2023,2,12,13,CLE,CIN,0,0,100,...,0,NaN,0,Week 1,CIN,3.0,CLE,24.0,NaN,True
12005,2023090700,09/07/2023,1,6,21,DET,KC,2,6,59,...,0,NaN,0,Week 1,DET,21.0,KC,20.0,NaN,False
12006,2023091000,09/10/2023,4,0,0,ATL,CAR,0,0,100,...,0,NaN,0,Week 1,CAR,10.0,ATL,24.0,NaN,True
12007,2023091000,09/10/2023,4,2,0,CAR,ATL,0,0,100,...,0,NaN,0,Week 1,CAR,10.0,ATL,24.0,NaN,True
12008,2023091000,09/10/2023,2,2,0,ATL,CAR,0,0,100,...,0,NaN,0,Week 1,CAR,10.0,ATL,24.0,NaN,True


In [8]:
avg_points_scored_home = all_data.groupby('Home')['HomeScore'].mean()
avg_points_scored_visitor = all_data.groupby('Visitor')['VisitorScore'].mean()

In [9]:
avg_points_allowed_home = all_data.groupby('Home')['VisitorScore'].mean()
avg_points_allowed_visitor = all_data.groupby('Visitor')['HomeScore'].mean()

In [10]:
overall_avg_points_scored = (avg_points_scored_home + avg_points_scored_visitor) / 2
overall_avg_points_allowed = (avg_points_allowed_home + avg_points_allowed_visitor) / 2

In [11]:
home_wins = all_data.groupby('Home')['HomeWon'].sum()
visitor_wins = all_data.groupby('Visitor').apply(lambda x: len(x) - x['HomeWon'].sum())

In [12]:
total_games_home = all_data['Home'].value_counts()
total_games_visitor = all_data['Visitor'].value_counts()

In [13]:
overall_wins = home_wins + visitor_wins
total_games = total_games_home + total_games_visitor

In [14]:
win_rate = overall_wins / total_games

In [15]:
team_features = pd.DataFrame({
    'AvgPointsScored': overall_avg_points_scored,
    'AvgPointsAllowed': overall_avg_points_allowed,
    'WinRate': win_rate
})

In [16]:
team_features.reset_index(inplace=True)
team_features.rename(columns={'Home': 'Team'}, inplace=True)

In [17]:
team_features.head()

,Team,AvgPointsScored,AvgPointsAllowed,WinRate
0,ARI,21.249213,26.051803,0.281280
1,ATL,20.206864,22.968158,0.419031
2,BAL,23.524597,18.131403,0.692794
3,BUF,29.256235,16.520392,0.757371
4,CAR,19.355687,23.957818,0.262437


In [19]:
upcomingGames.head()

,Home,Visitor
0,CHI,WAS
1,HOU,ATL
2,CAR,DET
3,NO,NE
4,CIN,ARI


In [20]:
all_data['SuccessfulPlay'] = all_data['IsTouchdown'] | (~all_data['IsInterception'] & ~all_data['IsFumble'])

In [21]:
all_data.head()

,GameId,Date,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,PenaltyType,PenaltyYards,Week,Visitor,VisitorScore,Home,HomeScore,OT,HomeWon,SuccessfulPlay
0,2022091106,09/11/2022,2,1,17,MIA,NE,2,10,55,...,NaN,0,Week 1,NE,7.0,MIA,20.0,NaN,True,-1
1,2022091106,09/11/2022,3,7,4,NE,MIA,4,1,72,...,ILLEGAL CONTACT,5,Week 1,NE,7.0,MIA,20.0,NaN,True,-1
2,2022091106,09/11/2022,3,6,57,NE,MIA,1,10,77,...,NaN,0,Week 1,NE,7.0,MIA,20.0,NaN,True,-1
3,2022091106,09/11/2022,3,6,17,NE,MIA,2,10,77,...,NaN,0,Week 1,NE,7.0,MIA,20.0,NaN,True,-1
4,2022091106,09/11/2022,4,9,46,NE,MIA,4,3,100,...,OFFENSIVE HOLDING,0,Week 1,NE,7.0,MIA,20.0,NaN,True,-1


In [22]:
avg_conceded_plays_home = all_data.groupby('Home')['SuccessfulPlay'].mean()

In [23]:
avg_conceded_plays_visitor = all_data.groupby('Visitor')['SuccessfulPlay'].mean()

In [24]:
overall_avg_conceded_plays = (avg_conceded_plays_home + avg_conceded_plays_visitor) / 2

In [25]:
all_data['Turnover'] = all_data['IsInterception'] | all_data['IsFumble']
avg_forced_turnovers_home = all_data.groupby('Home')['Turnover'].mean()
avg_forced_turnovers_visitor = all_data.groupby('Visitor')['Turnover'].mean()
overall_avg_forced_turnovers = (avg_forced_turnovers_home + avg_forced_turnovers_visitor) / 2

In [26]:
team_features_defensive = pd.DataFrame({
    'Team': team_features['Team'].values,
    'AvgPointsDefended': team_features['AvgPointsAllowed'].values,
    'AvgConcededPlays': overall_avg_conceded_plays.values,
    'AvgForcedTurnovers': overall_avg_forced_turnovers.values
})

In [27]:
team_features_combined = team_features.merge(team_features_defensive, on='Team')

In [28]:
team_features_combined.head()

,Team,AvgPointsScored,AvgPointsAllowed,WinRate,AvgPointsDefended,AvgConcededPlays,AvgForcedTurnovers
0,ARI,21.249213,26.051803,0.281280,26.051803,-1.023127,0.026036
1,ATL,20.206864,22.968158,0.419031,22.968158,-1.020005,0.021061
2,BAL,23.524597,18.131403,0.692794,18.131403,-1.024222,0.024222
3,BUF,29.256235,16.520392,0.757371,16.520392,-1.026120,0.027544
4,CAR,19.355687,23.957818,0.262437,23.957818,-1.019339,0.021912


In [29]:
avg_yards_per_play_home = all_data.groupby('Home')['Yards'].mean()
avg_yards_per_play_visitor = all_data.groupby('Visitor')['Yards'].mean()
overall_avg_yards_per_play = (avg_yards_per_play_home + avg_yards_per_play_visitor) / 2

In [30]:
total_yards_per_game_home = all_data.groupby(['SeasonYear', 'Home'])['Yards'].sum() / all_data.groupby(['SeasonYear', 'Home']).size()
total_yards_per_game_visitor = all_data.groupby(['SeasonYear', 'Visitor'])['Yards'].sum() / all_data.groupby(['SeasonYear', 'Visitor']).size()
overall_avg_yards_per_game = (total_yards_per_game_home + total_yards_per_game_visitor).groupby(level=1).mean()

In [31]:
avg_pass_completion_rate_home = all_data.groupby('Home').apply(lambda x: 1 - x['IsIncomplete'].mean())
avg_pass_completion_rate_visitor = all_data.groupby('Visitor').apply(lambda x: 1 - x['IsIncomplete'].mean())
overall_avg_pass_completion_rate = (avg_pass_completion_rate_home + avg_pass_completion_rate_visitor) / 2

In [32]:
avg_touchdowns_per_game_home = all_data.groupby(['SeasonYear', 'Home'])['IsTouchdown'].sum() / all_data.groupby(['SeasonYear', 'Home']).size()
avg_touchdowns_per_game_visitor = all_data.groupby(['SeasonYear', 'Visitor'])['IsTouchdown'].sum() / all_data.groupby(['SeasonYear', 'Visitor']).size()
overall_avg_touchdowns_per_game = (avg_touchdowns_per_game_home + avg_touchdowns_per_game_visitor).groupby(level=1).mean()

In [33]:
avg_rush_success_rate_home = all_data.groupby('Home').apply(lambda x: x['Yards'][x['IsRush'] == 1].mean())
avg_rush_success_rate_visitor = all_data.groupby('Visitor').apply(lambda x: x['Yards'][x['IsRush'] == 1].mean())
overall_avg_rush_success_rate = (avg_rush_success_rate_home + avg_rush_success_rate_visitor) / 2

In [34]:
new_offensive_features = pd.DataFrame({
    'Team': team_features_combined['Team'],
    'AvgYardsPerPlay': overall_avg_yards_per_play.values,
    'AvgYardsPerGame': overall_avg_yards_per_game.values,
    'AvgPassCompletionRate': overall_avg_pass_completion_rate.values,
    'AvgTouchdownsPerGame': overall_avg_touchdowns_per_game.values,
    'AvgRushSuccessRate': overall_avg_rush_success_rate.values
})

In [35]:
team_features_expanded = team_features_combined.merge(new_offensive_features, on='Team')

In [36]:
team_features_expanded.head()

,Team,AvgPointsScored,AvgPointsAllowed,WinRate,AvgPointsDefended,AvgConcededPlays,AvgForcedTurnovers,AvgYardsPerPlay,AvgYardsPerGame,AvgPassCompletionRate,AvgTouchdownsPerGame,AvgRushSuccessRate
0,ARI,21.249213,26.051803,0.281280,26.051803,-1.023127,0.026036,4.242580,8.294894,0.862756,0.062749,4.861024
1,ATL,20.206864,22.968158,0.419031,22.968158,-1.020005,0.021061,4.385171,8.129579,0.873059,0.053437,4.743080
2,BAL,23.524597,18.131403,0.692794,18.131403,-1.024222,0.024222,4.185915,7.886954,0.866361,0.051536,4.699528
3,BUF,29.256235,16.520392,0.757371,16.520392,-1.026120,0.027544,4.549029,8.825019,0.849617,0.065753,5.120168
4,CAR,19.355687,23.957818,0.262437,23.957818,-1.019339,0.021912,4.087563,7.788390,0.857721,0.054450,4.707913


In [37]:
avg_yards_allowed_per_play_home = all_data.groupby('Home')['Yards'].mean()
avg_yards_allowed_per_play_visitor = all_data.groupby('Visitor')['Yards'].mean()
overall_avg_yards_allowed_per_play = (avg_yards_allowed_per_play_home + avg_yards_allowed_per_play_visitor) / 2

In [38]:
total_yards_allowed_per_game_home = all_data.groupby(['SeasonYear', 'Home'])['Yards'].sum() / all_data.groupby(['SeasonYear', 'Home']).size()
total_yards_allowed_per_game_visitor = all_data.groupby(['SeasonYear', 'Visitor'])['Yards'].sum() / all_data.groupby(['SeasonYear', 'Visitor']).size()
overall_avg_yards_allowed_per_game = (total_yards_allowed_per_game_home + total_yards_allowed_per_game_visitor).groupby(level=1).mean()

In [39]:
avg_pass_completion_allowed_rate_home = all_data.groupby('Home').apply(lambda x: 1 - x['IsIncomplete'].mean())
avg_pass_completion_allowed_rate_visitor = all_data.groupby('Visitor').apply(lambda x: 1 - x['IsIncomplete'].mean())
overall_avg_pass_completion_allowed_rate = (avg_pass_completion_allowed_rate_home + avg_pass_completion_allowed_rate_visitor) / 2

In [40]:
avg_touchdowns_allowed_per_game_home = all_data.groupby(['SeasonYear', 'Home'])['IsTouchdown'].sum() / all_data.groupby(['SeasonYear', 'Home']).size()
avg_touchdowns_allowed_per_game_visitor = all_data.groupby(['SeasonYear', 'Visitor'])['IsTouchdown'].sum() / all_data.groupby(['SeasonYear', 'Visitor']).size()
overall_avg_touchdowns_allowed_per_game = (avg_touchdowns_allowed_per_game_home + avg_touchdowns_allowed_per_game_visitor).groupby(level=1).mean()

In [41]:
avg_rush_success_allowed_rate_home = all_data.groupby('Home').apply(lambda x: x['Yards'][x['IsRush'] == 1].mean())
avg_rush_success_allowed_rate_visitor = all_data.groupby('Visitor').apply(lambda x: x['Yards'][x['IsRush'] == 1].mean())
overall_avg_rush_success_allowed_rate = (avg_rush_success_allowed_rate_home + avg_rush_success_allowed_rate_visitor) / 2

In [42]:
new_defensive_features = pd.DataFrame({
    'Team': team_features_expanded['Team'],
    'AvgYardsAllowedPerPlay': overall_avg_yards_allowed_per_play.values,
    'AvgYardsAllowedPerGame': overall_avg_yards_allowed_per_game.values,
    'AvgPassCompletionAllowedRate': overall_avg_pass_completion_allowed_rate.values,
    'AvgTouchdownsAllowedPerGame': overall_avg_touchdowns_allowed_per_game.values,
    'AvgRushSuccessAllowedRate': overall_avg_rush_success_allowed_rate.values
})

In [43]:
team_features_complete = team_features_expanded.merge(new_defensive_features, on='Team')

team_features_complete

,Team,AvgPointsScored,AvgPointsAllowed,WinRate,AvgPointsDefended,AvgConcededPlays,AvgForcedTurnovers,AvgYardsPerPlay,AvgYardsPerGame,AvgPassCompletionRate,AvgTouchdownsPerGame,AvgRushSuccessRate,AvgYardsAllowedPerPlay,AvgYardsAllowedPerGame,AvgPassCompletionAllowedRate,AvgTouchdownsAllowedPerGame,AvgRushSuccessAllowedRate
0,ARI,21.249213,26.051803,0.281280,26.051803,-1.023127,0.026036,4.242580,8.294894,0.862756,0.062749,4.861024,4.242580,8.294894,0.862756,0.062749,4.861024
1,ATL,20.206864,22.968158,0.419031,22.968158,-1.020005,0.021061,4.385171,8.129579,0.873059,0.053437,4.743080,4.385171,8.129579,0.873059,0.053437,4.743080
2,BAL,23.524597,18.131403,0.692794,18.131403,-1.024222,0.024222,4.185915,7.886954,0.866361,0.051536,4.699528,4.185915,7.886954,0.866361,0.051536,4.699528
3,BUF,29.256235,16.520392,0.757371,16.520392,-1.026120,0.027544,4.549029,8.825019,0.849617,0.065753,5.120168,4.549029,8.825019,0.849617,0.065753,5.120168
4,CAR,19.355687,23.957818,0.262437,23.957818,-1.019339,0.021912,4.087563,7.788390,0.857721,0.054450,4.707913,4.087563,7.788390,0.857721,0.054450,4.707913
5,CHI,20.347226,27.921273,0.160361,27.921273,-1.028150,0.029883,4.496126,8.665669,0.885568,0.075307,5.218843,4.496126,8.665669,0.885568,0.075307,5.218843
6,CIN,23.316057,21.077524,0.600796,21.077524,-1.019392,0.020062,4.270071,7.978124,0.843282,0.051580,4.473877,4.270071,7.978124,0.843282,0.051580,4.473877
7,CLE,21.891552,22.726809,0.410239,22.726809,-1.019959,0.021776,4.216679,7.873387,0.848002,0.055621,4.917492,4.216679,7.873387,0.848002,0.055621,4.917492
8,DAL,28.810259,17.598943,0.713266,17.598943,-1.023195,0.026047,4.197979,7.954357,0.867374,0.057716,4.796513,4.197979,7.954357,0.867374,0.057716,4.796513
9,DEN,17.941634,23.139933,0.271373,23.139933,-1.021003,0.022306,4.072937,8.357621,0.848203,0.066001,4.749451,4.072937,8.357621,0.848203,0.066001,4.749451


In [44]:
upcomingGames = pd.read_csv("./Data/BEST New_Upcoming_Schedule (1).csv")

In [45]:
upcoming_encoded_home = upcomingGames.merge(team_features_complete, left_on='Home', right_on='Team', how='left')
upcoming_encoded_both = upcoming_encoded_home.merge(team_features_complete, left_on='Visitor', right_on='Team', suffixes=('_Home', '_Visitor'), how='left')

In [46]:
upcoming_encoded_both

,Home,Visitor,Team_Home,AvgPointsScored_Home,AvgPointsAllowed_Home,WinRate_Home,AvgPointsDefended_Home,AvgConcededPlays_Home,AvgForcedTurnovers_Home,AvgYardsPerPlay_Home,...,AvgYardsPerPlay_Visitor,AvgYardsPerGame_Visitor,AvgPassCompletionRate_Visitor,AvgTouchdownsPerGame_Visitor,AvgRushSuccessRate_Visitor,AvgYardsAllowedPerPlay_Visitor,AvgYardsAllowedPerGame_Visitor,AvgPassCompletionAllowedRate_Visitor,AvgTouchdownsAllowedPerGame_Visitor,AvgRushSuccessAllowedRate_Visitor
0,CHI,WAS,CHI,20.347226,27.921273,0.160361,27.921273,-1.028150,0.029883,4.496126,...,3.996204,7.716750,0.851979,0.055710,4.384546,3.996204,7.716750,0.851979,0.055710,4.384546
1,HOU,ATL,HOU,18.650102,23.404583,0.174369,23.404583,-1.024909,0.026876,4.226157,...,4.385171,8.129579,0.873059,0.053437,4.743080,4.385171,8.129579,0.873059,0.053437,4.743080
2,CAR,DET,CAR,19.355687,23.957818,0.262437,23.957818,-1.019339,0.021912,4.087563,...,4.566240,8.600770,0.850298,0.069365,4.566709,4.566240,8.600770,0.850298,0.069365,4.566709
3,NO,NE,NO,19.258114,22.119352,0.358287,22.119352,-1.019505,0.020930,4.115105,...,4.092565,7.790817,0.858085,0.049399,4.417244,4.092565,7.790817,0.858085,0.049399,4.417244
4,CIN,ARI,CIN,23.316057,21.077524,0.600796,21.077524,-1.019392,0.020062,4.270071,...,4.242580,8.294894,0.862756,0.062749,4.861024,4.242580,8.294894,0.862756,0.062749,4.861024
5,NYJ,DEN,NYJ,18.796860,19.590481,0.426084,19.590481,-1.022988,0.024761,4.027295,...,4.072937,8.357621,0.848203,0.066001,4.749451,4.072937,8.357621,0.848203,0.066001,4.749451
6,DAL,SF,DAL,28.810259,17.598943,0.713266,17.598943,-1.023195,0.026047,4.197979,...,4.399121,8.436793,0.853411,0.059129,4.456944,4.399121,8.436793,0.853411,0.059129,4.456944
7,JAX,BUF,JAX,23.387894,23.338547,0.444663,23.338547,-1.022699,0.024030,4.522797,...,4.549029,8.825019,0.849617,0.065753,5.120168,4.549029,8.825019,0.849617,0.065753,5.120168
8,TEN,IND,TEN,18.441639,20.096517,0.487846,20.096517,-1.020024,0.021082,4.180384,...,4.005052,7.782164,0.868932,0.055648,4.244982,4.005052,7.782164,0.868932,0.055648,4.244982
9,NYG,MIA,NYG,19.460300,25.042957,0.487349,25.042957,-1.021128,0.021457,4.186830,...,4.747582,9.852519,0.853976,0.094634,4.886131,4.747582,9.852519,0.853976,0.094634,4.886131


In [47]:
for col in ['AvgPointsScored', 'AvgPointsAllowed', 'WinRate', 'AvgPointsDefended', 'AvgConcededPlays', 'AvgForcedTurnovers',
            'AvgYardsPerPlay', 'AvgYardsPerGame', 'AvgPassCompletionRate', 'AvgTouchdownsPerGame', 'AvgRushSuccessRate',
            'AvgYardsAllowedPerPlay', 'AvgYardsAllowedPerGame', 'AvgPassCompletionAllowedRate', 'AvgTouchdownsAllowedPerGame', 'AvgRushSuccessAllowedRate']:
    upcoming_encoded_both[f'Diff_{col}'] = upcoming_encoded_both[f'{col}_Home'] - upcoming_encoded_both[f'{col}_Visitor']

In [48]:
upcoming_encoded_final = upcoming_encoded_both[['Home', 'Visitor'] + [col for col in upcoming_encoded_both.columns if 'Diff_' in col]]

upcoming_encoded_final

,Home,Visitor,Diff_AvgPointsScored,Diff_AvgPointsAllowed,Diff_WinRate,Diff_AvgPointsDefended,Diff_AvgConcededPlays,Diff_AvgForcedTurnovers,Diff_AvgYardsPerPlay,Diff_AvgYardsPerGame,Diff_AvgPassCompletionRate,Diff_AvgTouchdownsPerGame,Diff_AvgRushSuccessRate,Diff_AvgYardsAllowedPerPlay,Diff_AvgYardsAllowedPerGame,Diff_AvgPassCompletionAllowedRate,Diff_AvgTouchdownsAllowedPerGame,Diff_AvgRushSuccessAllowedRate
0,CHI,WAS,0.132698,5.275714,-0.417640,5.275714,-0.007968,0.006542,0.499922,0.948919,0.033589,0.019597,0.834297,0.499922,0.948919,0.033589,0.019597,0.834297
1,HOU,ATL,-1.556762,0.436426,-0.244661,0.436426,-0.004905,0.005814,-0.159013,-0.010995,-0.013997,0.002666,-0.199504,-0.159013,-0.010995,-0.013997,0.002666,-0.199504
2,CAR,DET,-6.874320,-0.764202,-0.293118,-0.764202,-0.002002,0.002306,-0.478677,-0.812380,0.007423,-0.014916,0.141204,-0.478677,-0.812380,0.007423,-0.014916,0.141204
3,NO,NE,-0.441334,1.764766,-0.082782,1.764766,0.003057,-0.005743,0.022540,-0.148223,-0.009632,-0.003137,0.243131,0.022540,-0.148223,-0.009632,-0.003137,0.243131
4,CIN,ARI,2.066844,-4.974278,0.319515,-4.974278,0.003735,-0.005974,0.027491,-0.316770,-0.019474,-0.011169,-0.387147,0.027491,-0.316770,-0.019474,-0.011169,-0.387147
5,NYJ,DEN,0.855226,-3.549452,0.154711,-3.549452,-0.001985,0.002455,-0.045642,-0.671782,-0.009988,-0.015872,-0.210990,-0.045642,-0.671782,-0.009988,-0.015872,-0.210990
6,DAL,SF,2.784259,2.545704,-0.070747,2.545704,-0.001427,0.002577,-0.201142,-0.482436,0.013963,-0.001413,0.339569,-0.201142,-0.482436,0.013963,-0.001413,0.339569
7,JAX,BUF,-5.868341,6.818155,-0.312708,6.818155,0.003422,-0.003514,-0.026232,-0.333284,0.000350,-0.002838,-0.538695,-0.026232,-0.333284,0.000350,-0.002838,-0.538695
8,TEN,IND,-0.298141,-3.326530,0.044606,-3.326530,0.005008,-0.005879,0.175332,-0.006562,-0.014921,-0.009482,-0.014654,0.175332,-0.006562,-0.014921,-0.009482,-0.014654
9,NYG,MIA,-9.180025,0.982457,-0.166916,0.982457,-0.000299,-0.001123,-0.560751,-2.256059,0.005010,-0.036478,0.077494,-0.560751,-2.256059,0.005010,-0.036478,0.077494


In [49]:
training_encoded_home = all_data.merge(team_features_complete, left_on='Home', right_on='Team', how='left')
training_encoded_both = training_encoded_home.merge(team_features_complete, left_on='Visitor', right_on='Team', suffixes=('_Home', '_Visitor'), how='left')

In [50]:
for col in ['AvgPointsScored', 'AvgPointsAllowed', 'WinRate', 'AvgPointsDefended', 'AvgConcededPlays', 'AvgForcedTurnovers',
            'AvgYardsPerPlay', 'AvgYardsPerGame', 'AvgPassCompletionRate', 'AvgTouchdownsPerGame', 'AvgRushSuccessRate',
            'AvgYardsAllowedPerPlay', 'AvgYardsAllowedPerGame', 'AvgPassCompletionAllowedRate', 'AvgTouchdownsAllowedPerGame', 'AvgRushSuccessAllowedRate']:
    training_encoded_both[f'Diff_{col}'] = training_encoded_both[f'{col}_Home'] - training_encoded_both[f'{col}_Visitor']

In [52]:
training_data = training_encoded_both[[col for col in training_encoded_both.columns if 'Diff_' in col]]
training_labels = training_encoded_both['HomeWon']

In [53]:
training_data.head()

,Diff_AvgPointsScored,Diff_AvgPointsAllowed,Diff_WinRate,Diff_AvgPointsDefended,Diff_AvgConcededPlays,Diff_AvgForcedTurnovers,Diff_AvgYardsPerPlay,Diff_AvgYardsPerGame,Diff_AvgPassCompletionRate,Diff_AvgTouchdownsPerGame,Diff_AvgRushSuccessRate,Diff_AvgYardsAllowedPerPlay,Diff_AvgYardsAllowedPerGame,Diff_AvgPassCompletionAllowedRate,Diff_AvgTouchdownsAllowedPerGame,Diff_AvgRushSuccessAllowedRate
0,8.940877,3.705914,0.213196,3.705914,0.001733,-0.004094,0.655017,2.061703,-0.004109,0.045236,0.468887,0.655017,2.061703,-0.004109,0.045236,0.468887
1,8.940877,3.705914,0.213196,3.705914,0.001733,-0.004094,0.655017,2.061703,-0.004109,0.045236,0.468887,0.655017,2.061703,-0.004109,0.045236,0.468887
2,8.940877,3.705914,0.213196,3.705914,0.001733,-0.004094,0.655017,2.061703,-0.004109,0.045236,0.468887,0.655017,2.061703,-0.004109,0.045236,0.468887
3,8.940877,3.705914,0.213196,3.705914,0.001733,-0.004094,0.655017,2.061703,-0.004109,0.045236,0.468887,0.655017,2.061703,-0.004109,0.045236,0.468887
4,8.940877,3.705914,0.213196,3.705914,0.001733,-0.004094,0.655017,2.061703,-0.004109,0.045236,0.468887,0.655017,2.061703,-0.004109,0.045236,0.468887


In [54]:
training_data.shape

(50607, 16)

In [55]:
training_labels.head()

0    True
1    True
2    True
3    True
4    True
Name: HomeWon, dtype: bool

In [56]:
logreg = LogisticRegression(max_iter=1000)

In [58]:
cross_val_scores = cross_val_score(logreg, training_data, training_labels, cv=5)

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py", line 1196, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 584, in _validate_data
    X, y = check_X_y(X, y, **check_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py", line 1106, in check_X_y
    X = check_arra

In [59]:
nan_columns = training_data.columns[training_data.isna().any()].tolist()

In [60]:
nan_counts = training_data[nan_columns].isna().sum()
nan_counts

Diff_AvgPointsScored                 3288
Diff_AvgPointsAllowed                3288
Diff_WinRate                         3288
Diff_AvgPointsDefended               3288
Diff_AvgConcededPlays                3288
Diff_AvgForcedTurnovers              3288
Diff_AvgYardsPerPlay                 3288
Diff_AvgYardsPerGame                 3288
Diff_AvgPassCompletionRate           3288
Diff_AvgTouchdownsPerGame            3288
Diff_AvgRushSuccessRate              3288
Diff_AvgYardsAllowedPerPlay          3288
Diff_AvgYardsAllowedPerGame          3288
Diff_AvgPassCompletionAllowedRate    3288
Diff_AvgTouchdownsAllowedPerGame     3288
Diff_AvgRushSuccessAllowedRate       3288
dtype: int64

In [61]:
training_data_cleaned = training_data.dropna()
training_labels_cleaned = training_labels.loc[training_data_cleaned.index]

In [62]:
training_data_cleaned.shape, training_labels_cleaned.shape

((47319, 16), (47319,))

In [63]:
cross_val_scores_cleaned = cross_val_score(logreg, training_data_cleaned, training_labels_cleaned, cv=5)
cross_val_scores_cleaned_mean = cross_val_scores_cleaned.mean()
cross_val_scores_cleaned_mean

0.7104121993484613

In [64]:
logreg.fit(training_data_cleaned, training_labels_cleaned)

LogisticRegression(max_iter=1000)

In [65]:
upcoming_game_probabilities = logreg.predict_proba(upcoming_encoded_final[[col for col in upcoming_encoded_final.columns if 'Diff_' in col]])

In [66]:
upcoming_game_probabilities

array([[0.91313479, 0.08686521],
       [0.7292858 , 0.2707142 ],
       [0.80366699, 0.19633301],
       [0.57108307, 0.42891693],
       [0.14126281, 0.85873719],
       [0.28387934, 0.71612066],
       [0.5539365 , 0.4460635 ],
       [0.78686483, 0.21313517],
       [0.43721279, 0.56278721],
       [0.66933257, 0.33066743],
       [0.23496121, 0.76503879],
       [0.04535241, 0.95464759],
       [0.31811167, 0.68188833],
       [0.4398352 , 0.5601648 ]])

In [67]:
upcoming_game_prob_home_win = upcoming_game_probabilities[:, 1]

In [68]:
upcoming_encoded_final['HomeWinProbability'] = upcoming_game_prob_home_win

/var/folders/qr/v5x52x113cg3d83cf7s0d2tc0000gn/T/ipykernel_2551/1546551890.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upcoming_encoded_final['HomeWinProbability'] = upcoming_game_prob_home_win


In [69]:
upcoming_predictions = upcoming_encoded_final[['Home', 'Visitor', 'HomeWinProbability']].sort_values(by='HomeWinProbability', ascending=False)

upcoming_predictions

,Home,Visitor,HomeWinProbability
11,PHI,LA,0.954648
4,CIN,ARI,0.858737
10,BAL,PIT,0.765039
5,NYJ,DEN,0.716121
12,KC,MIN,0.681888
8,TEN,IND,0.562787
13,GB,LV,0.560165
6,DAL,SF,0.446064
3,NO,NE,0.428917
9,NYG,MIA,0.330667
